In [123]:
import os
from itertools import product, permutations
import subprocess
import time

# Generation Queens ipynb

### Main class

In [124]:
class generation_Queens:
    
    def __init__(self, boardsize,  nb_restriction = None):
        self.boardsize = boardsize
        self.nb_queens = boardsize
        self.ymax = boardsize
        self.xmax = boardsize
        self.ymin = 1
        self.xmin = 1
        
        if(nb_restriction):
        
            if(nb_restriction == 1):
                self.positions = self.generate_all_positions_restriction1()
            if(nb_restriction == 2):
                self.positions = self.generate_all_positions_restriction2()
                 
    def pred_open(self, modif1=None, modif2=None):

        _modif1 = (
            str(modif1) if modif1 is not None and modif1 < 0 else "+" + str(modif1)
        )
        _modif2 = (
            str(modif2) if modif2 is not None and modif2 < 0 else "+" + str(modif2)
        )

        if modif1 is None and modif2 is None:
            return "open(?x,?y)"
        elif modif1 is None and modif2 is not None:
            return "open(?x,?y" + _modif2 + ")"
        elif modif1 is not None and modif2 is None:
            return "open(?x" + _modif1 + ",?y)"
        elif modif1 is not None and modif2 is not None:
            return "open(?x" + _modif1 + ",?y" + _modif2 + ")"

        assert "Error in open function, not case matched"

    def generation_preconditions_pos_queen(self, pos_x, pos_y):        
        str_return = []

        # top / down
        for _modif in range(self.ymin, self.ymax + 1, 1):
            # top
            if pos_y + _modif <= self.ymax:
                str_return.append(self.pred_open(None, _modif))
            # down
            if pos_y - _modif >= self.ymin:
                str_return.append(self.pred_open(None, -_modif))

        # left / right
        for _modif in range(self.xmin, self.xmax + 1, 1):
            # right
            if pos_x + _modif <= self.xmax:
                str_return.append(self.pred_open(_modif, None))
            # left
            if pos_x - _modif >= self.xmin:
                str_return.append(self.pred_open(-_modif, None))

        # diagnoal : the only relevant cases to check are the following
        # +-----+----+-----+
        # | TL  |    | TR  |
        # +-----+----+-----+
        # |     | x  |     |
        # +-----+----+-----+
        # | BL  |    | BR  |
        # +-----+----+-----+

        for _modif in range(self.xmin, self.xmax + 1, 1):
            # TL : top left
            if pos_x - _modif >= self.xmin and pos_y + _modif <= self.ymax:
                str_return.append(self.pred_open(-_modif, _modif))
            # BL : bottom left
            if pos_x - _modif >= self.xmin and pos_y - _modif >= self.ymin:
                str_return.append(self.pred_open(-_modif, -_modif))
            # TR : top right
            if pos_x + _modif <= self.xmax and pos_y + _modif <= self.ymax:
                str_return.append(self.pred_open(_modif, _modif))
            # BR : bottom right
            if pos_x + _modif <= self.xmax and pos_y - _modif >= self.ymin:
                str_return.append(self.pred_open(_modif, -_modif))

        return str_return
     
    def generate_function(self, pos_x, pos_y, color):

        preconditions = self.generation_preconditions_pos_queen(pos_x, pos_y)

        str_function = ":action setQueen" + str(pos_x) + str(pos_y) + "\n"
        str_function += ":parameters (?x,?y)\n"
        str_function += ":precondition (" + " ".join(preconditions) + ")\n"
        str_function += ":effect (" + color + "(?x,?y))"

        return str_function
    
    def logical_or(self, a, b):
        return "not(not(" + a + ") not(" + b + "))"

    def generation_preconditions(self):
        str_functions = []

        for i in range(self.xmin, self.xmax + 1, 1):
            for j in range(self.ymin, self.ymax + 1, 1):
                str_functions.append(self.generation_preconditions_pos_queen(i, j))

        return str_functions

    def generation_goal_states(self):
        str_goal = []

        for i in range(self.xmin, self.xmax + 1, 1):
            for j in range(self.ymin, self.ymax + 1, 1):
                str_goal.append("black(" + str(i) + "," + str(j) + ")")

        return str_goal

    def generate_all_positions_restriction1(self):
        
        valid_positions_xy = []
        for x in range(1, self.boardsize + 1):
            for y in range(1, self.boardsize + 1):
                valid_positions_xy.append((x ,y))  # Convert to 1-based index
                
        self.positions = valid_positions_xy
        return self.positions

    def generate_all_positions_restriction2(self):

        row_permutations = permutations(range(1, self.boardsize + 1))  # Generate row permutations (1-based index)
    
        # Use a set to store unique (x, y) positions
        unique_positions = set()
        for row_placement in row_permutations:
            for col, row in enumerate(row_placement):
                unique_positions.add((col + 1, row))  # Store (x, y) positions

        return list(unique_positions)

    def generate_domain(self):
        str_domain = ""

        # blackactions
        str_domain += "#blackactions\n"

        for x, y in self.positions:
            str_domain += self.generate_function(x, y, "black")
            str_domain += "\n"

        # whiteactions
        str_domain += "#whiteactions\n"

        function_white = ":action doNothing\n"
        function_white += ":parameters (?x, ?y)\n"
        function_white += ":precondition (open(?x,?y))\n"
        function_white += ":effect (white(?x,?y))\n"

        str_domain += function_white
        
        with open(f"./tmp/domain_{self.boardsize}x{self.boardsize}.ig", "w", encoding='utf-8') as file:
                file.write(str_domain)

    def generate_problem(self, problem_filename, Qs, forbidden = []):
        
        ########### preprocessing ###########
        Qs = [Q for Q in Qs if Q is not None]
        str_known_positions = " ".join([f'black({x},{y})' for (x,y) in Qs])
        forbidden_positions = " ".join([f'open({x},{y})' for (x,y) in forbidden]) if len(forbidden) > 0 else ""
        remaining_positions = list(set(self.positions) - set(Qs) - set(forbidden))
        
        ########### baordsize + depth  ###########
        str_problem = "#boardsize\n"
        str_problem += str(self.boardsize) + " " + str(self.boardsize) + "\n"
        str_problem += "#depth\n"
        str_problem += str(self.boardsize) + "\n"
       
        ########### times ###########
        str_problem += "#times\n"
        
        nb_times = self.boardsize - len(Qs) 
        str_problem += " ".join(['t'+str(2*i + 1) for i in range(nb_times)]) + '\n'
        
        ########### init ###########
        str_problem += "#init\n"
        str_problem += str_known_positions+ "\n"
        
        ########### blackgoal ########### 
        str_problem += "#blackgoal\n"   
        
        for(x,y) in remaining_positions:
            str_problem += f'{str_known_positions} black({x},{y}) {forbidden_positions}\n'
        
        ########### white goal  ###########
        str_problem += "#whitegoal\n"
        str_problem += "white(3,1)"

        ########### write to file  ########
        filename = f"./tmp/problem_{self.boardsize}x{self.boardsize}.ig" if problem_filename is None else problem_filename
        with open(f"./tmp/{filename}.ig", "w", encoding='utf-8') as file:
                file.write(str_problem)
                
    def generate_problem2(self, problem_filename, initial, goal, forbidden = []):
        
        ########### preprocessing ###########
        str_init_positions = " ".join([f'black({x},{y})' for (x,y) in initial]) 
        str_known_positions = " ".join([f'black({x},{y})' for (x,y) in goal])
        forbidden_positions = " ".join([f'black({x},{y})' for (x,y) in forbidden]) if len(forbidden) > 0 else ""
        remaining_positions = list(set(self.positions) - set(initial))
        
        ########### baordsize + depth  ###########
        str_problem = "#boardsize\n"
        str_problem += str(self.boardsize) + " " + str(self.boardsize) + "\n"
        str_problem += "#depth\n"
        str_problem += str(self.boardsize) + "\n"
       
        ########### times ###########
        str_problem += "#times\n"
        
        nb_times = self.boardsize - len(initial) 
        str_problem += " ".join(['t'+str(2*i + 1) for i in range(nb_times)]) + '\n'
        
        ########### init ###########
        str_problem += "#init\n"
        str_problem += str_init_positions+ "\n"
        
        ########### blackgoal ########### 
        str_problem += "#blackgoal\n"   
        str_problem += str_known_positions + " " +  forbidden_positions + '\n'
        
        ########### white goal  ###########
        str_problem += "#whitegoal\n"
        str_problem += "white(3,1)"

        ########### write to file  ########
        filename = f"./tmp/problem_{self.boardsize}x{self.boardsize}.ig" if problem_filename is None else problem_filename
        with open(f"./tmp/{filename}.ig", "w", encoding='utf-8') as file:
                file.write(str_problem) 

In [125]:
def run_q_sage(problem, domain, print_output = False):
    # run Q-Sage.py with the appropriate files + arguments
    project_root = (
        subprocess.run(["pwd"], capture_output=True, text=True)
        .stdout[:-2]
        .split("/")[:-1]
    )
    project_root = "/".join(project_root)

    args = [
        "--game_type",
        "general",
        "--ib_problem",
        problem,
        "--ib_domain",
        domain,
        "-e",
        "ib",
        "--run",
        "2",
        "--encoding_out",
        "intermediate_files/encoding/NQueens.txt",
    ]
      
    result = subprocess.run(
        ["python", "Q-sage.py"] + args,
        cwd=project_root,
        capture_output=True,
        text=True
    )
    
    # processing result 
    result = result.stdout
    if(print_output):
        print(result)
    
    i = result.find('setQueen')
     
    if(i != -1):
        l = len('setQueen')
        return (int(result[i+l]), int(result[i+l+1]))
    else:
        return None

In [126]:
def visualise_boad(boardsize, black_queens):
    # Create a board with a border
    board = [['.' for _ in range(boardsize)] for _ in range(boardsize)]
    
    # Place the queens on the board
    qs = [q for q in black_queens if q is not None]
    for x, y in qs:
        board[x-1][y-1] = 'Q'
    
    # Print the board in reverse so that (1,1) is bottom left 
    for row in reversed(board):
        print(" ".join(row))

In [127]:
def backtracking(Q, current, visited, solutions, prefix = ""):
    print(current,visited,solutions)
    # file name semantics 
    domain_file = f"NQueens/tmp/domain_{Q.boardsize}x{Q.boardsize}.ig"
    problem_file = f"NQueens/tmp/problem_{Q.boardsize}x{Q.boardsize}_"+prefix
    name = f"problem_{Q.boardsize}x{Q.boardsize}_"+prefix
    i = len(current)
    
    forbidden = list(set(visited) - set(current)) if len(visited) > 0 else []
    Q.generate_problem(name+'_t'+str(i), current, forbidden)
    new_pos = run_q_sage(problem_file, domain_file)
   
    if(new_pos != False):

        if(len(current) == Q.boardsize):
            current.append(new_pos)
            
            visited.append(new_pos)
            solutions.append(current)
            print("Solution found")
            visualise_boad(Q.boardsize, current)
        else:
            backtracking(Q, current, visited, solutions, prefix)
    else : 
        if(len(current) == Q.boardsize):
            print("No solution found")
#Q = generation_Queens(4,1)
#backtracking(Q, [(1,2)], [], [])

In [129]:
# generalize to function 
def find_a_solution(Q, positions, prefix, debug = False ):

    domain_file = f"NQueens/tmp/domain_{Q.boardsize}x{Q.boardsize}.ig"
    l = "NQueens/tmp/"
    name = f"problem_{Q.boardsize}x{Q.boardsize}_"+prefix
    
    i = 1   
    forbidden = {i: [] for i in range(0, Q.boardsize + 1)}
    
    while Q.boardsize != len(positions) and len(positions) != 0:  
        if(debug):
            print(f"positions : {positions}, forbidden :  {forbidden[len(positions)]}, i : {i}, forbidden all : {forbidden}")

        Q.generate_problem(name+'_t'+str(i), positions, forbidden[len(positions)])
        newPos = run_q_sage(f"{l+name}_t{str(i)}.ig", domain_file) 
        
        if(debug):
            print('position found : ', newPos)
        
        if(newPos == None):                
            tmp = positions.pop()
            forbidden[len(positions)].append(tmp)
           
        else : 
            positions.append(newPos)
        
        i+=1 
            
    print("These are the positions that were found", positions)
    visualise_boad(Q.boardsize, positions)
    return False if None in positions or len(positions) == 0 else positions

In [131]:
Q_nqueens_test = generation_Queens(4, 1)
debug_mode = False 
assert find_a_solution(Q_nqueens_test, [(1,1)], 'find_a_solution_incorrect', debug_mode) == False , "Test failed, problem in solve_nqueens function"	
assert len(list(set(find_a_solution(Q_nqueens_test, [(1,2)], 'find_a_solution_correct', debug_mode)) - set([(1, 2), (2, 4), (3, 1), (4, 3)]))) == 0, "Test failed, problem in solve_nqueens function"

These are the positions that were found []
. . . .
. . . .
. . . .
. . . .
These are the positions that were found [(1, 2), (2, 4), (4, 3), (3, 1)]
. . Q .
Q . . .
. . . Q
. Q . .


In [ ]:
Q_nqueens_test = generation_Queens(4, 2)
assert solve_nqueens(Q_nqueens_test, (1,1), 'incorrect') == False , "Test failed, problem in solve_nqueens function"	
#assert solve_nqueens(Q_nqueens_test, (1,2), 'correct') == [(1, 2), (2, 4), (3, 1), (4, 3)], "Test failed, problem in solve_nqueens function"

In [ ]:
Q_nqueens_test2 = generation_Queens(5, 2)
solve_nqueens(Q_nqueens_test2, (1,1), '_findMistake')
solve_nqueens(Q_nqueens_test2, (1,1), '_findMistake')

## Give next position

In [ ]:
def next_position(boardsize, positions, restrictions = 1 ):
    
    Q = generation_Queens(boardsize, restrictions)
    Q.generate_domain()
    
    domain_file = f"NQueens/tmp/domain_{Q.boardsize}x{Q.boardsize}.ig"
    problem_file = f"problem_{Q.boardsize}x{Q.boardsize}_guess_next_position"
    

    Q.generate_problem(problem_file, positions) 
    
    result = run_q_sage(f"NQueens/tmp/{problem_file}.ig", domain_file)
    if(result != None):
        return result
    

positions = [(1,2), (2,4), (3,1)]
next_pos = next_position(4, positions, 1)
print(f"Next position for the 4x4 board is {next_pos} having already placed the queens at {' '.join([str(pos) for pos in positions])}")
visualise_boad(4, positions + [next_pos])

## Validator

In [ ]:
def validator(proposed_solution, restrictions = 1, forbidden = []):
    
    boardsize = len(proposed_solution)
    Q = generation_Queens(boardsize, restrictions)
    Q.generate_domain()
    
    domain_file = f"NQueens/tmp/domain_{Q.boardsize}x{Q.boardsize}.ig"
    problem_file = f"problem_{Q.boardsize}x{Q.boardsize}_validator"
    
    # initial = proposed_solution[-1]
    # goal = proposed_solution 
    Q.generate_problem2(problem_file, proposed_solution[:-1], proposed_solution, forbidden)
    
    result = run_q_sage(f"NQueens/tmp/{problem_file}.ig", domain_file)
    if(result != None):
        print("The solution is valid")
        return True 
    else:
        print("The solution is not valid")
        return False

In [ ]:
assert validator([(1, 2), (2, 4), (3, 1), (4, 3)], 1) == True, "Test failed, problem in validator function"
assert validator([(1, 2), (2, 4), (3, 1), (4, 4)], 1) == False, "Test failed, problem in validator function"

## Generate all solutions 

In [ ]:
# generating all possible solutions for a board 
def generate_all_solutions(board_size, nb_restriction,root):
    
    Q = generation_Queens(board_size, nb_restriction)
    Q.generate_domain()
    
    positions = [(x, y) for x in range(1, board_size + 1) for y in range(1, board_size + 1)]

    succes = []
    for i, pos1 in enumerate(positions):
            print(f"Solution {i+1} : {pos1}")
            r = solve_nqueens(Q, pos1, f"all_{i+1}")
            print(r)
            if(r != False):
                succes.append(r)
            else : 
                for _i in range(1,board_size):
                    os.remove(f"{root}/NQueens/tmp/problem_{board_size}x{board_size}_all_{i+1}_t{_i}.ig")
            
    return succes

In [ ]:
project_root = (
        subprocess.run(["pwd"], capture_output=True, text=True)
        .stdout[:-2]
        .split("/")[:-1]
    )
project_root = "/".join(project_root)
project_root

In [ ]:
%%capture captured_output

start_time = time.time()
s = generate_all_solutions(4, 2, project_root)
end_time = time.time()

execution_time = end_time - start_time
minutes, seconds = divmod(execution_time, 60)
with open(f"{project_root}/NQueens/tmp/solution_4x4_output.txt", "w") as file:
    file.write(f"Execution time: {int(minutes)} minutes and {seconds:.2f} seconds\n")
    file.write(f"Solutions: {s}\nOutput: \n")
    file.write(captured_output.stdout)


In [ ]:
%%capture captured_output

start_time = time.time()
s = generate_all_solutions(5, 2, project_root)
end_time = time.time()

execution_time = end_time - start_time
minutes, seconds = divmod(execution_time, 60)
with open(f"{project_root}/NQueens/tmp/solution_5x5_output.txt", "w") as file:
    file.write(f"Execution time: {int(minutes)} minutes and {seconds:.2f} seconds\n")
    file.write(f"Solutions: {s}\nOutput: \n")
    file.write(captured_output.stdout)


In [ ]:
# check solution for 5x5 board
solutions_5x5 = [
    [(1, 1), (2, 3), (3, 5), (4, 2), (5, 4)],
    [(1, 1), (2, 4), (3, 2), (4, 5), (5, 3)],
    [(1, 2), (2, 4), (3, 1), (4, 3), (5, 5)],
    [(1, 2), (2, 5), (3, 3), (4, 1), (5, 4)],
    [(1, 3), (2, 1), (3, 4), (4, 2), (5, 5)],
    [(1, 3), (2, 5), (3, 2), (4, 4), (5, 1)],
    [(1, 4), (2, 1), (3, 3), (4, 5), (5, 2)],
    [(1, 4), (2, 2), (3, 5), (4, 3), (5, 1)],
    [(1, 5), (2, 2), (3, 4), (4, 1), (5, 3)],
    [(1, 5), (2, 3), (3, 1), (4, 4), (5, 2)],
]

solutions_5x5_found = [
    [(1, 2), (3, 3), (4, 1), (2, 5), (5, 4)],
    [(1, 3), (3, 4), (5, 5), (2, 1), (4, 2)],
    [(2, 1), (5, 2), (4, 5), (3, 3), (1, 4)],
    [(2, 3), (4, 4), (5, 2), (1, 5), (3, 1)],
    [(2, 4), (4, 5), (4, 5), (4, 5), (4, 5)],
    [(2, 5), (3, 3), (4, 1), (5, 4), (1, 2)],
    [(3, 3), (4, 1), (1, 2), (2, 5), (5, 4)],
    [(3, 4), (5, 5), (2, 1), (1, 3), (4, 2)],
    [(4, 1), (3, 3), (1, 2), (5, 4), (2, 5)],
    [(4, 2), (5, 4), (2, 3), (1, 1), (3, 5)],
    [(4, 3), (4, 3), (4, 3), (4, 3), (4, 3)],
    [(5, 1), (4, 3), (1, 4), (3, 5), (2, 2)],
    [(5, 2), (2, 1), (3, 3), (1, 4), (4, 5)],
    [(5, 3), (2, 4), (4, 5), (1, 1), (3, 2)],
    [(5, 5), (1, 3), (2, 1), (4, 2), (3, 4)],
]

valid_solutions = []
for solution in solutions_5x5_found:
    if set(solution) in [set(sol) for sol in solutions_5x5]:
        if any(set(solution) == set(valid_solution) for valid_solution in valid_solutions):
            continue
        else : 
            print(f"Solution {solution} is valid.")
            valid_solutions.append(solution)
    else:
        print(f"Solution {solution} is not valid.")

assert len(valid_solutions) == len(
    solutions_5x5
), f"Test failed {len(valid_solutions)} != {len(solutions_5x5)}, problem in generate_all_solutions function"

In [ ]:
solutions_6x6 = [
    [(1, 2), (2, 4), (3, 6), (4, 1), (5, 3), (6, 5)],
    [(1, 3), (2, 6), (3, 2), (4, 5), (5, 1), (6, 4)],
    [(1, 4), (2, 1), (3, 5), (4, 2), (5, 6), (6, 3)],
    [(1, 5), (2, 3), (3, 1), (4, 6), (5, 4), (6, 2)],
]

#### Application
Creating the instance of a boardsize queen problem, can be done by calling the constructor with either one of the restrictions. 
- restriction 1 : Queens can't be in the same column
- restriction 2 : Queens can't be in the same column + row 

In [ ]:
Q = generation_Queens(4,2)

#### Test (debugging purposes)

In [ ]:
Q = generation_Queens(4, 2)
Q.generate_domain()

solve_nqueens(Q, (1,3), 'correct')

solve_nqueens(Q, (2,2), 'incorrect')

In [ ]:
board_size = 4

Q = generation_Queens(board_size, 2)
Q.generate_domain()

pos1= (2,2)
Q.generate_problem('problem_4x4_incorrect_t1', [pos1])
pos2 = run_q_sage("NQueens/tmp/problem_4x4_incorrect_t1.ig", "NQueens/tmp/domain_4x4.ig") 

Q.generate_problem('problem_4x4_incorrect_t2', [pos1,pos2])
pos3 = run_q_sage("NQueens/tmp/problem_4x4_incorrect_t2.ig", "NQueens/tmp/domain_4x4.ig") 

Q.generate_problem('problem_4x4_incorrect_t3', [pos1,pos2,pos3])
pos4 = run_q_sage("NQueens/tmp/problem_4x4_incorrect_t3.ig", "NQueens/tmp/domain_4x4.ig") 


incorrect_result = [pos1, pos2, pos3, pos4] 
if(None in incorrect_result):
    print("One of the positions could not be found")
else : 
    print("The positions of the queens are : ", incorrect_result)

visualise_boad(board_size, [pos1, pos2, pos3, pos4])

In [ ]:
Q1 = generation_Queens(4)
unique_positions = Q1.generate_all_positions_restriction2()
flattened_positions = [pos for pos in unique_positions]

# Check if all positions are unique
assert len(flattened_positions) == len(set(flattened_positions)), "Positions are not unique"
# Check if all positions are encoded as (x, y)
assert all(isinstance(pos, tuple) and len(pos) == 2 for pos in flattened_positions), "Not all positions are encoded as (x, y)"

print("All positions are unique and the flattened list is:")
print(flattened_positions)

In [ ]:
Q2 = generation_Queens(4)
unique_positions = Q2.generate_all_positions_restriction1()
flattened_positions2 = [pos for pos in unique_positions]

# Check if all positions are unique
assert len(flattened_positions2) == len(set(flattened_positions2)), "Positions are not unique" + str(flattened_positions2[0:10])
assert all(isinstance(pos, tuple) and len(pos) == 2 for pos in flattened_positions2), "Not all positions are encoded as (x, y)"


print("All positions are unique and the flattened list is:")
print(flattened_positions2)

In [ ]:
#  Test if the run_q_sage finds an valid answer 
assert run_q_sage("NQueens/tmp/problem_4x4_3correct.ig", "NQueens/tmp/domain_4x4.ig") == (4,3) , "Error in run_q_sage function with 3 known positions"
assert run_q_sage("NQueens/tmp/problem_4x4_2correct.ig", "NQueens/tmp/domain_4x4.ig") == (3,1), "Error in run_q_sage function with 2 known positions"
assert run_q_sage("NQueens/tmp/problem_4x4_1correct.ig", "NQueens/tmp/domain_4x4.ig") == (2,4), "Error in run_q_sage function with 1 known position"

In [ ]:

Q_pos = generation_Queens(4)
r = Q_pos.generation_preconditions_pos_queen(2, 2)
r1 = Q_pos.generation_preconditions_pos_queen(1, 1)
r2 = Q_pos.generation_preconditions_pos_queen(3, 2)

assert len(r) == 11, "Length should be 11 but is {}".format(len(r), print(r))
assert len(r1) == 9, "Length should be 9 but is {}".format(len(r1), print(r1))
assert len(r2) == 11, "Length should be 11 but is {}".format(len(r2), print(r2))
print("Tests passed")

r_all = []
for i in range(1, 5):
    for j in range(1, 5):
        r_all.append(Q_pos.generation_preconditions_pos_queen(i, j))

# Flatten the nested list r_all
flattened_r_all = [item for sublist in r_all for item in sublist]

# Check the number of clauses
num_clauses = len(flattened_r_all)
total_of_clauses = 12 * 9 + 4 * 11
assert total_of_clauses == num_clauses, (
    "Total of clauses should be "
    + str(total_of_clauses)
    + " but is {}".format(num_clauses)
)